## Monte Carlo 

Monte Carlo Method is one of the computational technique mainly relies on random sampling to estimate the numerical method or probabilities widely used in problems that have uncertainity ,optimization and approximation 



in Reinforcement Learning it is a model free algorithm that mainly rely on sampled experience to estimate the value functions and optimize the policies 

Core Idea 
*It uses randomness which is deterministic in nature.
*it generates large numbers of random sample using average to approx the quantities or say estimate the value of the value function and update the policies 



Mathematical Intuition
*The law of large numbers is a mathematical concept that states that the average of a large number of independent random variables converging to expected value


State value function Vπ(s)=Eπ[Gt∣St=s]
which is expected return starting from state s under policy 𝜋.


Action-value function:
Qπ(s,a)=Eπ[Gt∣St=s,A t=a]
the expected return starting from state s, taking action a,and following π.

and the total return is defined by 
Gt=Rt+1+γ(Rt+2)+γ^2(Rt+1)+....γ^n(Rt+n+1)
where γ is the discounted factor ranging from [0,1]

## Algorithm for MC Control
Initialize 
*Q(s,a) arbitrarily and a policy π.
*Repeat for many episodes:

*Generate an episode by following π.

*For each state-action pair (s,a) in the episode:Compute the return 𝐺𝑡.

*Update Q(s,a) as the average of all returns observed after


Q(s,a)←Q(s,a)+α(Gt−Q(s,a)).
*Update π(s) as the greedy policy with respect to Q(s,a).



In [1]:
import numpy as np 
import random
import seaborn as sn 
import matplotlib.pyplot as plt


In [2]:
class MDP_Grid:
    def __init__(self,grid_size,start_state,end_state,rewards,discount_factor=0.9):
        self.grid_size = grid_size
        self.start_state = start_state
        self.end_state = end_state
        self.rewards=rewards
        self.discount_factor = discount_factor


    def transition_states(self,state,action):
        row,col=state
        # Transition to next state based on the action taken
        if action == 'up':
              next_state = (max(0, row - 1), col)
        elif action == 'down':
            next_state = (min(self.grid_size - 1, row + 1), col)
        elif action == 'left':
            next_state = (row, max(0, col - 1))
        elif action == 'right':
            next_state = (row, min(self.grid_size - 1, col + 1))
        
        # Reward checks from the Grid World
        reward = self.rewards.get((state, action), 0)
        return next_state, reward


In [ ]:
#Monte carlo Implementation
class Monte_carlo:
    def __init__(self,grid_size,start_state,end_state,discount_factor=0.9,epsilon=0.1):
        self.grid_size = grid_size
        self.start_state = start_state
        self.end_state = end_state
        self.discount_factor = discount_factor
        self.epsilon = epsilon

        #initialize the policy with Random experience
        self.policy = {(row,col):random.choice(['up','down','left','right'])
                       for row in range(grid_size) for col in range(grid_size) 
                       if(row,col)!=self.start_state and (row,col)!=self.end_state}
        

        #state action value
        self.q_val={}
        for row in range(grid_size):
            for col in range(grid_size):
                state=(row,col)
                if state!=self.start_state and state!=self.end_state:
                    self.q_val[state]={action:0 for action in ['up','down','left','right']}

        
        #Counting state-action pair occurrences for returning the average returns
        self.counts={}
        for row in range(grid_size):
            for col in range(grid_size):
                state=(row,col)
                if state!=self.start_state and state!=self.end_state:
                    self.counts[state]={action:0 for action in ['up','down','left','right']}

            
        def generate_episode(self,decision_model):
            episode=[]
            state=self.start_state
            while state!=self.end_state:
                if state not in self.policy:
                    break
                action=self.policy[state]
                next_state,reward =self.transition(state,action,decision_model)
                episode.append((state,action,next_state,reward))
                state=next_state
            return episode
        
        def update_policy(self):
            for state in self.q_val:
                for action in self.q_val[state]:
                    if self.counts[state][action]==0:
                        self.policy[state]=random.choice(['up','down','left','right'])
                    else:
                        self.policy[state]=max(self.q_val[state],key=self.q_val[state].get)

        def mc_update
                        

        
        